### Required packages

In [ ]:
!pip install neural-structured-learning

In [ ]:
!pip install split-folders

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
!pip install -q pyyaml h5py

### Imports

In [ ]:
## from __future__ import absolute_import
import matplotlib.pyplot as plt
import neural_structured_learning as nsl
import numpy as np
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import array_to_img, img_to_array
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
import pandas as pd
# from tensorflow.keras.utils import np_utils
from sklearn import preprocessing
from tensorflow.keras.applications.densenet import DenseNet121
import random
import math
import datetime


In [ ]:
from art.defences import AdversarialTrainer
from art.attacks import ProjectedGradientDescent
from art.classifiers import TensorFlowClassifier
from art.classifiers import TFClassifier, KerasClassifier
from art.attacks import FastGradientMethod


In [2]:
print(tf.__version__)

2.1.0


### Functions to load the data

In [3]:
def check_string_in_list(match_list, string):
    """Return matching string fro   m a list of possible matches"""

    # for possible_match in match_list:
    for possible_match in match_list:
        if possible_match in string:
            return possible_match

    return 'No matching label in list of possible matches'

In [4]:
def load_images_and_labels(rootdir, possible_labels):
    """

    Load and shuffle   the images and the labels from a directory. Assumes labels are given in the filenames.

    rootdir (str) : the directory where the images are stored
    possible_labels (list) : a list containing the possible labels of the task

    """
    loaded_images = list()
    labels = list()

    for subdir, dirs, files in os.walk(rootdir):
        for filename in files:
            image = Image.open(subdir + '/' + filename)#.convert('L')
            # image to array
#             pixels = img_to_array(image)
            pixels = image.resize((256, 256)) 
            # store loaded image
            loaded_images.append(np.asarray(pixels))
            # find label in filename and store label
            labels.append(check_string_in_list(possible_labels, filename))

#     labels = to_categorical(labels)
    # normalize the images
    le = preprocessing.LabelEncoder()
    labels = le.fit_transform(labels)
    loaded_images = np.asarray(loaded_images)
    loaded_images = (loaded_images - 127.5) / 127.5
    labels = np.asarray(labels)
    
    # shuffle the images and labels
    indices = np.arange(loaded_images.shape[0])
    np.random.shuffle(indices)

    loaded_images = loaded_images[indices]
    labels = labels[indices]
    
    print('Loaded {} images succesfully'.format(len(loaded_images)))
    return loaded_images, labels



### Define some parameters

In [31]:
NUM_SAMPLES = 2907
EPOCHS = 20
BATCH_SIZE = 16
FOLDS= 2
LEARN_RATE=0.0001
LAYER_UNITS = (64, 16)
TARGET_SIZE = (256, 256)
TRAIN_SIZE = .7
VAL_SIZE = .15
TEST_SIZE = .15
NUM_CLASSES = 6
NUM_TRAIN_SAMPLES = TRAIN_SIZE * NUM_SAMPLES
NUM_VAL_SAMPLES = VAL_SIZE * NUM_SAMPLES
ROOTDIR = 'C:/Users/trist/Documents/GitHub/DeepLearningProject/dataset-split'
possible_labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

Split data into train, validation, and test sets

In [ ]:
import split_folders
split_folders.ratio('C:/Users/trist/Documents/GitHub/DeepLearningProject/dataset-resized', output="dataset-split", seed=1337, ratio=(TRAIN_SIZE, VAL_SIZE, TEST_SIZE)) # default values

### Building the base model and define the callbacks

In [36]:
def build_base_model(input_shape=(256, 256, 3)):
     # build the VGG16 network
    from tensorflow.keras.models import Model
    densenet = DenseNet121(weights='imagenet',
                               include_top=False,
                               input_shape=input_shape)
    
    for layer in densenet.layers:
        layer.trainable = False
    
    # make batch normalization layers trainable to prevent overfitting
    for layer in densenet.layers:
        if "BatchNormalization" in layer.__class__.__name__:
            layer.trainable = True
            
    x = densenet.output
    x = Flatten()(x)

    for num_units in LAYER_UNITS:
        x = Dense(num_units, activation='relu')(x)
        x = Dropout(0.4)(x)

    predictions = Dense(6, activation='softmax')(x)
    custom_model = Model(inputs=densenet.input, outputs=predictions)
   
    return custom_model

In [27]:
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, min_delta=1e-4, mode='min')
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    return [mcp_save, reduce_lr_loss]

### Create augmentation functions

In [8]:
#https://github.com/zhunzhong07/Random-Erasing/blob/master/transforms.py
#https://jkjung-avt.github.io/keras-image-cropping/

def wrap_generator(batches, augmentation_type):
    """Take as input a Keras ImageGen (Iterator) and generate more augmentations
    according to augmentor function.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_augmented = np.zeros((batch_x.shape[0], 256, 256, 3))
        for i in range(batch_x.shape[0]):
            if augmentation_type is 'erase':
                augmentor = RandomErasing()
                batch_augmented[i] = augmentor(batch_x[i])
            if augmentation_type is 'blend':
                augmentor = MixingImages()
                batch_augmented[i] = augmentor(batch_x[i], i, batch_x, batch_y)
        yield (batch_augmented, batch_y)
        
class RandomErasing(object):
    '''
    Class that performs Random Erasing in Random Erasing Data Augmentation by Zhong et al. 
    -------------------------------------------------------------------------------------
    probability: The probability that the operation will be performed.
    sl: min erasing area
    sh: max erasing area
    r1: min aspect ratio
    mean: erasing value
    -------------------------------------------------------------------------------------
    '''
    def __init__(self, probability = 0.5, sl = 0.02, sh = 0.4, r1 = 0.3, mean=[0.4914, 0.4822, 0.4465]):
        self.probability = probability
        self.mean = mean
        self.sl = sl
        self.sh = sh
        self.r1 = r1
       
    def __call__(self, img):

        if random.uniform(0, 1) > self.probability:
            return img

        for attempt in range(100):
            area = img.shape[1] * img.shape[2]
            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, 1/self.r1)

            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))

            if w < img.shape[2] and h < img.shape[1]:
                x1 = random.randint(0, img.shape[1] - h)
                y1 = random.randint(0, img.shape[2] - w)
                if img.shape[0] == 3:
                    img[0, x1:x1+h, y1:y1+w] = self.mean[0]
                    img[1, x1:x1+h, y1:y1+w] = self.mean[1]
                    img[2, x1:x1+h, y1:y1+w] = self.mean[2]
                else:
                    img[0, x1:x1+h, y1:y1+w] = self.mean[0]
                return img

        return img

    
class MixingImages(object):
    '''
    Class that performs Random mixing of images. 
    -------------------------------------------------------------------------------------
    probability: The probability that the operation will be performed.
    -------------------------------------------------------------------------------------
    '''
    def __init__(self, probability = 0.5, alpha=0.5):
        self.probability = probability
        self.alpha = alpha
       
    def __call__(self, img, i, batch_x, batch_y):

        if random.uniform(0, 1) > self.probability:
            return img
        
        options = [im for im, label in zip(batch_x, bach_y) if label == batch_y[i]]
        img_to_blend = random.choice(options)
        blended_img = Image.blend(img, img_to_blend, self.alpha) 

        return blended_img

### Defining the train function

In [22]:
def new_train(ROOTDIR, TARGET_SIZE, EPOCHS, BATCH_SIZE, LEARN_RATE, NUM_TRAIN_SAMPLES, NUM_VAL_SAMPLES, datagen, \
              augmentation_type=None):
    
    # https://www.mlprojecttutorials.com/image%20recognition/transfer/
    train_gen = datagen.flow_from_directory(
        ROOTDIR + '/train', 
        target_size=TARGET_SIZE, 
        batch_size=BATCH_SIZE
    )
    val_gen = datagen.flow_from_directory(
        ROOTDIR + '/val', 
        target_size=TARGET_SIZE, 
        batch_size=BATCH_SIZE
    )
    test_gen = datagen.flow_from_directory(
        ROOTDIR + '/test', 
        target_size=TARGET_SIZE, 
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    
    name_weights = "final_model" + "_weights.h5"
    callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)
#     tf.compat.v1.disable_eager_execution()

    optimizer = optimizers.Adam(lr=LEARN_RATE)
    
    if augmentation_type is 'erase' or augmentation_type is 'blend':
        train_generator = wrap_generator(train_gen, augmentation_type)
        val_generator = wrap_generator(val_gen, augmentation_type)

        
        model = build_base_model()
        model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                   metrics=['acc'])
      
        model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(NUM_TRAIN_SAMPLES // BATCH_SIZE), 
                    epochs=EPOCHS,
                    shuffle=True,
                    verbose=1,
                    validation_data = val_generator,
                    validation_steps = int(NUM_VAL_SAMPLES // BATCH_SIZE),
                    callbacks = callbacks)        

    else:
       
        model = build_base_model()
        model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                   metrics=['acc', 'AUC', 'Precision', 'Recall'])
      
        model.fit(
                    train_gen,
                    steps_per_epoch = int(NUM_TRAIN_SAMPLES // BATCH_SIZE), 
                    epochs=EPOCHS,
                    shuffle=True,
                    verbose=1,
                    validation_data = val_gen,
                    validation_steps = int(NUM_VAL_SAMPLES // BATCH_SIZE),
                    callbacks = callbacks)
        
    return model, test_gen

def evaluate_model(model, test_gen):    
    filenames = test_gen.filenames
    nb_samples = len(filenames)
    
    predictions = model.predict(test_gen, steps = nb_samples)
    true_labels = test_gen.classes
    
    y_true = true_labels
    y_pred = np.array([np.argmax(x) for x in predictions])

    test_acc = sum(y_true == y_pred) / len(y_true)
    print('Accuracy: {}'.format(test_acc))
    return test_acc

### Perform the experiments

Without any augmentation

In [32]:
datagen_none = ImageDataGenerator(
                rescale=1./255,
               )

model, test_gen = new_train(ROOTDIR, TARGET_SIZE, EPOCHS, BATCH_SIZE, LEARN_RATE, NUM_TRAIN_SAMPLES, NUM_VAL_SAMPLES, datagen_none, \
              augmentation_type=None)

model.save_weights('my_checkpoint')


result = evaluate_model(model, test_gen)

Found 2019 images belonging to 6 classes.
Found 504 images belonging to 6 classes.
Found 384 images belonging to 6 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 127 steps, validate for 27 steps
Epoch 1/20
127/127 [==============================] - 86s 678ms/step - loss: 1.6474 - acc: 0.3140 - AUC: 0.6969 - val_loss: 1.4344 - val_acc: 0.4583 - val_AUC: 0.7851
Epoch 2/20
127/127 [==============================] - 61s 483ms/step - loss: 1.0287 - acc: 0.6176 - AUC: 0.8974 - val_loss: 1.2146 - val_acc: 0.5370 - val_AUC: 0.8488
Epoch 3/20
127/127 [==============================] - 61s 481ms/step - loss: 0.6565 - acc: 0.7732 - AUC: 0.9615 - val_loss: 1.0612 - val_acc: 0.6157 - val_AUC: 0.8874
Epoch 4/20
127/127 [==============================] - 61s 480ms/step - loss: 0.4161 - acc: 0.8826 - AUC: 0.9874 - val_loss: 1.0022 - val_acc: 0.6227 - val_AUC: 0.9001
Epoch 5/20
127/127 [==============================] - 59s 468ms/step - loss: 0.2650 - acc: 0.9351 - AUC: 0.9965 - v

With simple augmentation

In [37]:
datagen_simple = ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True)

model, test_gen = new_train(ROOTDIR, TARGET_SIZE, EPOCHS, BATCH_SIZE, LEARN_RATE, NUM_TRAIN_SAMPLES, NUM_VAL_SAMPLES, datagen_simple, \
              augmentation_type='simple')

result = evaluate_model(model, test_gen)

Found 2019 images belonging to 6 classes.
Found 504 images belonging to 6 classes.
Found 384 images belonging to 6 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 127 steps, validate for 27 steps
Epoch 1/20
127/127 [==============================] - 199s 2s/step - loss: 1.9815 - acc: 0.2100 - AUC: 0.5655 - val_loss: 1.7069 - val_acc: 0.2801 - val_AUC: 0.6594
Epoch 2/20
127/127 [==============================] - 152s 1s/step - loss: 1.7306 - acc: 0.2566 - AUC: 0.6299 - val_loss: 1.5844 - val_acc: 0.4259 - val_AUC: 0.7562
Epoch 3/20
127/127 [==============================] - 152s 1s/step - loss: 1.6832 - acc: 0.2888 - AUC: 0.6576 - val_loss: 1.5138 - val_acc: 0.4560 - val_AUC: 0.7713
Epoch 4/20
127/127 [==============================] - 46s 358ms/step - loss: 1.5719 - acc: 0.3289 - AUC: 0.7203 - val_loss: 1.3798 - val_acc: 0.5255 - val_AUC: 0.8280
Epoch 5/20
127/127 [==============================] - 35s 274ms/step - loss: 1.5469 - acc: 0.3447 - AUC: 0.7344 - val_los

With Random Erasing

In [18]:
datagen_none = ImageDataGenerator(
                rescale=1./255,
               )

model, test_gen = new_train(ROOTDIR, TARGET_SIZE, EPOCHS, BATCH_SIZE, LEARN_RATE, NUM_TRAIN_SAMPLES, NUM_VAL_SAMPLES, datagen_none, \
              augmentation_type='erase')

result = evaluate_model(model, test_gen)

Found 2019 images belonging to 6 classes.
Found 504 images belonging to 6 classes.
Found 384 images belonging to 6 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 63 steps, validate for 13 steps
Epoch 1/10
63/63 [==============================] - 18s 293ms/step - loss: 1.3823 - acc: 0.5025 - val_loss: 1.2022 - val_acc: 0.5745
Epoch 2/10
63/63 [==============================] - 14s 216ms/step - loss: 0.8249 - acc: 0.7056 - val_loss: 1.0004 - val_acc: 0.5735
Epoch 3/10
63/63 [==============================] - 11s 181ms/step - loss: 0.5681 - acc: 0.8113 - val_loss: 1.0308 - val_acc: 0.6078
Epoch 4/10
63/63 [==============================] - 12s 190ms/step - loss: 0.4419 - acc: 0.8681 - val_loss: 0.9611 - val_acc: 0.6740
Epoch 5/10
63/63 [==============================] - 11s 173ms/step - loss: 0.3384 - acc: 0.8998 - val_loss: 1.1655 - val_acc: 0.6397
Epoch 6/10
63/63 [=========================

With blending

In [19]:
datagen_none = ImageDataGenerator(
                rescale=1./255,
               )

model, test_gen = new_train(ROOTDIR, TARGET_SIZE, EPOCHS, BATCH_SIZE, LEARN_RATE, NUM_TRAIN_SAMPLES, NUM_VAL_SAMPLES, datagen_none, \
              augmentation_type='blending')

result = evaluate_model(model, test_gen)

Found 2019 images belonging to 6 classes.
Found 504 images belonging to 6 classes.
Found 384 images belonging to 6 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 63 steps, validate for 13 steps
Epoch 1/10
63/63 [==============================] - 18s 285ms/step - loss: 1.1365 - acc: 0.6054 - val_loss: 0.9381 - val_acc: 0.6442
Epoch 2/10
63/63 [==============================] - 11s 178ms/step - loss: 0.4342 - acc: 0.8651 - val_loss: 0.8248 - val_acc: 0.6899
Epoch 3/10
63/63 [==============================] - 11s 177ms/step - loss: 0.1955 - acc: 0.9512 - val_loss: 0.7439 - val_acc: 0.7260
Epoch 4/10
63/63 [==============================] - 11s 179ms/step - loss: 0.1022 - acc: 0.9834 - val_loss: 0.7349 - val_acc: 0.7284
Epoch 5/10
63/63 [==============================] - 11s 182ms/step - loss: 0.0597 - acc: 0.9945 - val_loss: 0.7121 - val_acc: 0.7404
Epoch 6/10
63/63 [==============================] - 11s 170ms/step - loss: 0.0402 - acc: 0.9980 - val_loss: 0.7674 - va

Loading the data

In [ ]:
X, y = load_images_and_labels(rootdir, possible_labels)


In [ ]:
def train(X, y, epochs, batch_size, folds, gen, learn_rate, augmentation_type=None):
    print('creating folds')
    folds = list(StratifiedKFold(n_splits=folds, shuffle=True, random_state=1).split(X, y))
    print('started learning')

#     metrics = pd.DataFrame()
#     df.loc[len(df)] = [1,2,3]
    metrics = []
    
    for fold, (train_idx, val_idx) in enumerate(folds):
        
        print('\nFold ', fold)
        X_train_cv = X[train_idx]
        y_train_cv = y[train_idx]
        X_valid_cv = X[val_idx]
        y_valid_cv= y[val_idx]
        
        y_train_cv = np_utils.to_categorical(y_train_cv)
        y_valid_cv = np_utils.to_categorical(y_valid_cv)
        
        name_weights = "final_model_fold" + str(fold) + "_weights.h5"
        callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)
        
        model = build_base_model((y_train_cv[0]).shape[0])
        optimizer = optimizers.Adam(lr=0.0001)
        
        if augmentation_type is 'adverserial':
            model = wrap_adverserial(model)
            model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                       metrics=['acc'])
            model.fit(x={'input': X_train_cv, 'label': y_train_cv}, batch_size=batch_size)
        
        else:
            generator = gen.flow(X_train_cv, y_train_cv, batch_size = batch_size, )

            model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                       metrics=['acc'])
            
            print(model.summary())
            
            model.fit_generator(
                        generator,
                        steps_per_epoch=len(X_train_cv)/batch_size,
                        epochs=epochs,
                        shuffle=True,
                        verbose=1,
                        validation_data = (X_valid_cv, y_valid_cv),
                        callbacks = callbacks)

        evaluation = model.evaluate(X_valid_cv, y_valid_cv)
#         metrics = metrics.loc[len(df)] = evaluation
        print(evaluation)
        metrics.append(evaluation)

        
    return metrics
        

def wrap_adverserial(model):
    adv_config = nsl.configs.make_adv_reg_config(
        multiplier = 0.2,
        adv_step_size = 0.2,
        adv_grad_norm = 'infinity'
    )
    adv_model = nsl.keras.AdversarialRegularization(model,
                                            label_keys=['label'],
                                            adv_config=adv_config)
    return adv_model

        